In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cargar el conjunto completo de datos de entrenamiento y el de prueba
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')  # Cargar el conjunto completo de prueba

# Paso 1: Imputación de valores faltantes en columnas categóricas
categorical_columns = [
    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO'
]

for column in categorical_columns:
    mode_value = train_data[column].mode()[0]
    train_data[column] = train_data[column].fillna(mode_value)
    if column in test_data.columns:
        test_data[column] = test_data[column].fillna(mode_value)

# Definir las características (X) y la variable objetivo (y) para el conjunto de entrenamiento
X = train_data.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y = train_data['RENDIMIENTO_GLOBAL']

# Dividir el conjunto de entrenamiento en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el conjunto de características (X_test) para el conjunto de prueba
X_test = test_data.drop(columns=['ID'])

# Identificar columnas numéricas y categóricas para el pipeline
numeric_features = ['PERIODO']
categorical_features = categorical_columns

# Crear transformadores para las columnas numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

# Crear un preprocesador que aplique las transformaciones correspondientes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear un pipeline que incluya el preprocesador y el modelo Random Forest
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entrenar el modelo en el subconjunto de entrenamiento
pipeline.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = pipeline.predict(X_val)

# Evaluar el desempeño del modelo en el conjunto de validación
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Exactitud en el conjunto de validación: {accuracy:.4f}")
print("\nMatriz de Confusión en el conjunto de validación:")
print(confusion_matrix(y_val, y_val_pred))
print("\nReporte de Clasificación en el conjunto de validación:")
print(classification_report(y_val, y_val_pred))

# Realizar predicciones en el conjunto de prueba completo
all_predictions = pipeline.predict(X_test)

# Crear el DataFrame para el archivo de envío
submission = pd.DataFrame({
    'ID': test_data['ID'],  # Usar los IDs del conjunto de prueba completo
    'RENDIMIENTO_GLOBAL': all_predictions
})

# Guardar el archivo de envío en formato CSV
submission.to_csv('solucion.csv', index=False)
print("El archivo 'solucion.csv' ha sido generado y está listo para descargar.")

Exactitud en el conjunto de validación: 0.3599

Matriz de Confusión en el conjunto de validación:
[[17468  4927  7309  5461]
 [ 4519 14869  6184  9001]
 [ 9385  8518  8046  8310]
 [ 6198 11429  7413  9463]]

Reporte de Clasificación en el conjunto de validación:
              precision    recall  f1-score   support

        alto       0.46      0.50      0.48     35165
        bajo       0.37      0.43      0.40     34573
  medio-alto       0.28      0.23      0.25     34259
  medio-bajo       0.29      0.27      0.28     34503

    accuracy                           0.36    138500
   macro avg       0.35      0.36      0.35    138500
weighted avg       0.35      0.36      0.36    138500

El archivo 'solucion.csv' ha sido generado y está listo para descargar.
